In [ ]:
import os
import base64
import langchain
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
with open("../data/secrets_itor.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization

from chatrag.retriever import create_retriever_from_csv, create_retrieval_chain

In [ ]:
langchain.debug = True

## Data preparation

In [ ]:
retriever_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)
# retriever_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)
retriever = create_retriever_from_csv(
    csv_path="../data/movies_title_overview_vote.csv",
    metadata_columns_dtypes={"monthly_traffic": "int"},
    llm=retriever_llm
)

# Vectorstore DB

In [ ]:
retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
# retrieval_chain_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=2000)

In [ ]:
g_media_sq_retrieval_chain = create_retrieval_chain(retriever=retriever, llm=retrieval_chain_llm)

# Tooling

In [ ]:
calc_tool = load_tools(["llm-math"], llm=OpenAI(temperature=0))

In [ ]:
retriever_description = """Movie search tool. The action input must be just topics in a natural language sentence"""

In [ ]:
tools = [
    Tool(
        name = "Search movies",
        func=g_media_sq_retrieval_chain.run,
        description=retriever_description
    ),
    calc_tool[0]
]

# Prompt templates

In [ ]:
prefix = """ You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. 

For the first interactions you should collect some user configuration data. This data will restrict the movies to consider.

User Data to collect (mandatory):
    Target genre: Movie genre e.g. fiction, adventure, trhiller...
    Movie overview topic: List of keywords defining the campaign context.

After succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good as you can. To do so, you have access to the following tools:"""

In [ ]:
suffix = """Begin! Your first action must ve collect user data and keep it. Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation: ... Thought: ... Action: ..."""


In [ ]:
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": prefix,
        "suffix": suffix
    }

# Chains

In [ ]:
# agent_llm_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", max_tokens=1000)
agent_llm_model = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=2000)

In [ ]:
general_chat_agent = initialize_agent(
    tools=tools,
    llm=agent_llm_model,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    agent_kwargs=agent_kwargs,
)

# Stepwise

In [ ]:
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union
from langchain.utils.input import get_color_mapping
from langchain.schema import (
    AgentAction,
    AgentFinish)
import time
from IPython.core.debugger import set_trace

def _call_stepwise(
        self,
        inputs: Dict[str, str],
        run_manager = None,
    ) -> Dict[str, Any]:
        """Run text through and get agent response."""
        # Construct a mapping of tool name to tool for easy lookup
        name_to_tool_map = {tool.name: tool for tool in self.tools}
        # We construct a mapping from each tool to a color, used for logging.
        color_mapping = get_color_mapping(
            [tool.name for tool in self.tools], excluded_colors=["green", "red"]
        )
        intermediate_steps: List[Tuple[AgentAction, str]] = []
        # Let's start tracking the number of iterations and time elapsed
        iterations = 0
        time_elapsed = 0.0
        start_time = time.time()
        # We now enter the agent loop (until it returns something).
        while self._should_continue(iterations, time_elapsed):
            set_trace()
            next_step_output = self._take_next_step(
                name_to_tool_map,
                color_mapping,
                inputs,
                intermediate_steps,
                run_manager=run_manager,
            )
            if isinstance(next_step_output, AgentFinish):
                return self._return(
                    next_step_output, intermediate_steps, run_manager=run_manager
                )

            intermediate_steps.extend(next_step_output)
            if len(next_step_output) == 1:
                next_step_action = next_step_output[0]
                # See if tool should return directly
                tool_return = self._get_tool_return(next_step_action)
                if tool_return is not None:
                    return self._return(
                        tool_return, intermediate_steps, run_manager=run_manager
                    )
            iterations += 1
            time_elapsed = time.time() - start_time
        output = self.agent.return_stopped_response(
            self.early_stopping_method, intermediate_steps, **inputs
        )
        return self._return(output, intermediate_steps, run_manager=run_manager)

In [ ]:
inputs = general_chat_agent.prep_inputs("I want to watch a movie about outer space exploration.")
inputs

In [ ]:
# Construct a mapping of tool name to tool for easy lookup
name_to_tool_map = {tool.name: tool for tool in general_chat_agent.tools}
# We construct a mapping from each tool to a color, used for logging.
color_mapping = get_color_mapping(
    [tool.name for tool in general_chat_agent.tools], excluded_colors=["green", "red"]
)

In [ ]:
intermediate_steps: List[Tuple[AgentAction, str]] = []
# Let's start tracking the number of iterations and time elapsed
iterations = 0
time_elapsed = 0.0
# start_time = time.time()

In [ ]:

next_step_output = general_chat_agent._take_next_step(
    name_to_tool_map,
    color_mapping,
    inputs,
    intermediate_steps,
    run_manager=None,
)

In [ ]:
next_step_output

In [ ]:
print(next_step_output.log)

In [ ]:
next_step_output.return_values

In [ ]:
len(next_step_output)

In [ ]:
isinstance(next_step_output, AgentFinish)

In [ ]:
general_chat_agent.prep_outputs(inputs, next_step_output.return_values, False)

In [ ]:
general_chat_agent.memory

In [ ]:
inputs = general_chat_agent.prep_inputs("Thriller.")
inputs

In [ ]:
# Construct a mapping of tool name to tool for easy lookup
name_to_tool_map = {tool.name: tool for tool in general_chat_agent.tools}
# We construct a mapping from each tool to a color, used for logging.
color_mapping = get_color_mapping(
    [tool.name for tool in general_chat_agent.tools], excluded_colors=["green", "red"]
)

In [ ]:
intermediate_steps: List[Tuple[AgentAction, str]] = []
# Let's start tracking the number of iterations and time elapsed
iterations = 0
time_elapsed = 0.0
# start_time = time.time()

In [ ]:
next_step_output = general_chat_agent._take_next_step(
    name_to_tool_map,
    color_mapping,
    inputs,
    intermediate_steps,
    run_manager=None,
)

In [ ]:
isinstance(next_step_output, AgentFinish)

In [ ]:
next_step_output

In [ ]:
next_step_output

In [ ]:
len(next_step_output)

In [ ]:
next_step_output[0][0]

In [ ]:
print(next_step_output[0][0].log)

In [ ]:
next_step_output[0][1]

In [ ]:
intermediate_steps.extend(next_step_output)
intermediate_steps

In [ ]:
len(next_step_output) == 1

In [ ]:
next_step_action = next_step_output[0]
next_step_action

In [ ]:
tool_return = general_chat_agent._get_tool_return(next_step_action)
print(tool_return)

In [ ]:
{tool.name: tool for tool in general_chat_agent.tools}

In [ ]:
next_step_action[0].tool in name_to_tool_map

In [ ]:
name_to_tool_map[next_step_action[0].tool].return_direct

In [ ]:
iterations += 1
time_elapsed = time.time() - start_time

In [ ]:
general_chat_agent.agent.get_full_inputs(intermediate_steps)

In [ ]:
print(general_chat_agent.agent.get_full_inputs(intermediate_steps)["agent_scratchpad"])

In [ ]:
next_step_output = general_chat_agent._take_next_step(
    name_to_tool_map,
    color_mapping,
    inputs,
    intermediate_steps,
    run_manager=None,
)

In [ ]:
next_step_output

In [ ]:
isinstance(next_step_output, AgentFinish)

In [ ]:
print(next_step_output.log)

In [ ]:
general_chat_agent._return(next_step_output, intermediate_steps, run_manager=None)

## Reproduce plan step
With tool calling

In [ ]:
def plan(
    self,
    intermediate_steps: List[Tuple[AgentAction, str]],
    callbacks: Callbacks = None,
    **kwargs: Any,
) -> Union[AgentAction, AgentFinish]:
    """Given input, decided what to do.

    Args:
        intermediate_steps: Steps the LLM has taken to date,
            along with observations
        callbacks: Callbacks to run.
        **kwargs: User inputs.

    Returns:
        Action specifying what tool to use.
    """
    full_inputs = self.get_full_inputs(intermediate_steps, **kwargs)
    full_output = self.llm_chain.predict(callbacks=callbacks, **full_inputs)
    return self.output_parser.parse(full_output)

def predict(self, callbacks: Callbacks = None, **kwargs: Any) -> str:
    """Format prompt with kwargs and pass to LLM.

    Args:
        callbacks: Callbacks to pass to LLMChain
        **kwargs: Keys to pass to prompt template.

    Returns:
        Completion from LLM.

    Example:
        .. code-block:: python

            completion = llm.predict(adjective="funny")
    """
    return self(kwargs, callbacks=callbacks)[self.output_key]

def _call(
    self,
    inputs: Dict[str, Any],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> Dict[str, str]:
    response = self.generate([inputs], run_manager=run_manager)
    return self.create_outputs(response)[0]

def generate(
    self,
    input_list: List[Dict[str, Any]],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> LLMResult:
    """Generate LLM result from inputs."""
    prompts, stop = self.prep_prompts(input_list, run_manager=run_manager)
    return self.llm.generate_prompt(
        prompts,
        stop,
        callbacks=run_manager.get_child() if run_manager else None,
        **self.llm_kwargs,
    )

def generate_prompt(
    self,
    prompts: List[PromptValue],
    stop: Optional[List[str]] = None,
    callbacks: Callbacks = None,
    **kwargs: Any,
) -> LLMResult:
    prompt_messages = [p.to_messages() for p in prompts]
    return self.generate(prompt_messages, stop=stop, callbacks=callbacks, **kwargs)


def generate(
    self,
    messages: List[List[BaseMessage]],
    stop: Optional[List[str]] = None,
    callbacks: Callbacks = None,
    *,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    **kwargs: Any,
) -> LLMResult:
    """Top Level call"""
    params = self._get_invocation_params(stop=stop, **kwargs)
    options = {"stop": stop}

    callback_manager = CallbackManager.configure(
        callbacks,
        self.callbacks,
        self.verbose,
        tags,
        self.tags,
        metadata,
        self.metadata,
    )
    run_managers = callback_manager.on_chat_model_start(
        dumpd(self), messages, invocation_params=params, options=options
    )
    results = []
    for i, m in enumerate(messages):
        try:
            results.append(
                self._generate_with_cache(
                    m,
                    stop=stop,
                    run_manager=run_managers[i] if run_managers else None,
                    **kwargs,
                )
            )
        except (KeyboardInterrupt, Exception) as e:
            if run_managers:
                run_managers[i].on_llm_error(e)
            raise e
    flattened_outputs = [
        LLMResult(generations=[res.generations], llm_output=res.llm_output)
        for res in results
    ]
    llm_output = self._combine_llm_outputs([res.llm_output for res in results])
    generations = [res.generations for res in results]
    output = LLMResult(generations=generations, llm_output=llm_output)
    if run_managers:
        run_infos = []
        for manager, flattened_output in zip(run_managers, flattened_outputs):
            manager.on_llm_end(flattened_output)
            run_infos.append(RunInfo(run_id=manager.run_id))
        output.run = run_infos
    return output


def _generate(
    self,
    messages: List[BaseMessage],
    stop: Optional[List[str]] = None,
    run_manager: Optional[CallbackManagerForLLMRun] = None,
    stream: Optional[bool] = None,
    **kwargs: Any,
) -> ChatResult:
    if stream if stream is not None else self.streaming:
        generation: Optional[ChatGenerationChunk] = None
        for chunk in self._stream(
            messages=messages, stop=stop, run_manager=run_manager, **kwargs
        ):
            if generation is None:
                generation = chunk
            else:
                generation += chunk
        assert generation is not None
        return ChatResult(generations=[generation])

    message_dicts, params = self._create_message_dicts(messages, stop)
    params = {**params, **kwargs}
    response = self.completion_with_retry(
        messages=message_dicts, run_manager=run_manager, **params
    )
    return self._create_chat_result(response)

In [ ]:
full_inputs = general_chat_agent.agent.get_full_inputs([], **inputs)
full_inputs

In [ ]:
full_output = general_chat_agent.agent.llm_chain.predict(callbacks=None, **full_inputs)

In [ ]:
print(full_output)

In [ ]:
general_chat_agent.agent.output_parser.parse(full_output)

In [ ]:
print("System:  You are an assistant expert in movies recommendation. You should guide and help the user through the whole process until suggesting the best movie options to watch. You should attend to all the user requirements always taking into account user data. \n\nFor the first interactions you should collect some user configuration data. This data will restrict the movies to consider.\n\nUser Data to collect (mandatory):\n    Target genre: Movie genre e.g. fiction, adventure, trhiller...\n    Movie overview topic: List of keywords defining the campaign context.\n\nAfter succesfully collecting data, you should keep the conversation with the human, answering the questions and requests as good as you can. To do so, you have access to the following tools:\n\nSearch movies: Movie search tool. The action input must be just topics in a natural language sentence, args: {{'tool_input': {{'type': 'string'}}}}\nCalculator: Useful for when you need to answer questions about math., args: {{'tool_input': {{'type': 'string'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid \"action\" values: \"Final Answer\" or Search movies, Calculator\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{\n  \"action\": $TOOL_NAME,\n  \"action_input\": $INPUT\n}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{\n  \"action\": \"Final Answer\",\n  \"action_input\": \"Final response to human\"\n}\n```\n\nBegin! Your first action must ve collect user data and keep it. Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB``` then Observation: ... Thought: ... Action: ...\nHuman: I want to watch a movie about outer space exploration.\nAI: Sure, I can help with that. Could you please specify your preferred genre? For example, are you interested in science fiction, adventure, drama, or something else?\nHuman: Thriller.")

In [ ]:
prompts, stop = general_chat_agent.agent.llm_chain.prep_prompts([full_inputs])
prompts, stop

In [ ]:
# general_chat_agent.agent.generate(
#         [prompt],
#         stop=stop,
#         callbacks=callbacks,
#         tags=tags,
#         metadata=metadata,
#         **kwargs,
#     ).generations[0][0].text

In [ ]:
prompts[0].messages

In [ ]:
general_chat_agent.agent.llm_chain.llm.generate_prompt(
        prompts,
        stop,
        callbacks=None,
        **general_chat_agent.agent.llm_chain.llm_kwargs,
    )

In [ ]:
prompt_messages = [p.to_messages() for p in prompts]
prompt_messages

In [ ]:
prompt_messages = [p.to_messages() for p in prompts]
prompt_messages

In [ ]:
print(general_chat_agent.agent.llm_chain.llm.cache)

In [ ]:
print(langchain.llm_cache)

In [ ]:
for i, m in enumerate(prompt_messages):
    print(i)
    print(m)

In [ ]:
print(prompt_messages[0][3].content)

In [ ]:
stop

In [ ]:
general_chat_agent.agent.llm_chain.llm._generate(prompt_messages[0], stop=stop)

In [ ]:
message_dicts, params = general_chat_agent.agent.llm_chain.llm._create_message_dicts(prompt_messages[0], stop)
message_dicts, params

In [ ]:
params['top_p'] = 1

In [ ]:
response = general_chat_agent.agent.llm_chain.llm.completion_with_retry(messages=message_dicts, **params)
response

In [ ]:
response = general_chat_agent.agent.llm_chain.llm.completion_with_retry(messages=message_dicts, **params)
response

In [ ]:
chat_results = general_chat_agent.agent.llm_chain.llm._create_chat_result(response)
chat_results

In [ ]:
from langchain.schema import LLMResult

In [ ]:
flattened_outputs = [LLMResult(generations=[res.generations], llm_output=res.llm_output) for res in [chat_results]]
flattened_outputs

In [ ]:
llm_output = general_chat_agent.agent.llm_chain.llm._combine_llm_outputs([res.llm_output for res in [chat_results]])
generations = [res.generations for res in [chat_results]]

In [ ]:
llm_output, generations

In [ ]:
generate_output = LLMResult(generations=generations, llm_output=llm_output)
generate_output

In [ ]:
full_output_inner = general_chat_agent.agent.llm_chain.create_outputs(generate_output)[0]
full_output_inner

In [ ]:
general_chat_agent.agent.output_parser.parse

In [ ]:
# Porqué el parse ejecuta una LLM chain? Pensaba que era un StructuredChatOutputParserWithRetries y este no tiene nada de eso (creo)
output_parser_out = general_chat_agent.agent.output_parser.parse(full_output_inner)

In [ ]:
output_parser_out